# Advanced Querying Mongo

Importing libraries and setting up connection

In [1]:
# from pymongo import MongoClient
# client = MongoClient("")

In [2]:
%pip install pymongo

Note: you may need to restart the kernel to use updated packages.


In [3]:
from pymongo import MongoClient
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [4]:
cliente=MongoClient('mongodb://localhost:27017') 

cliente

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [5]:
# Your Code

In [6]:
cliente.list_database_names() 

['admin', 'companies', 'config', 'local']

In [7]:
db=cliente.companies

In [8]:
collection = db.companies

In [9]:
#list(collection.find())

In [10]:
def name_Babelgum():
    query=collection.find({'name': 'Babelgum'},    
                          {'name': 1, '_id':0}
                         )
    
    df=pd.json_normalize(data=query, meta=['name'])
    
    return df

In [11]:
name_Babelgum()

,name
0,Babelgum


### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [12]:
# Your Code

In [13]:
def more_fiveK_employees():
    query=db.companies.find({'number_of_employees': {'$gt': 5000}},    
                            {'name': 1, '_id':0, 'number_of_employees':1}
                           ).sort('number_of_employees', -1).limit(20)
    
    df=pd.json_normalize(data=query, meta=['name', 'number_of_employees'])
    
    return df

In [14]:
more_fiveK_employees()

,name,number_of_employees
0,Siemens,405000
1,IBM,388000
2,Toyota,320000
3,PayPal,300000
4,Nippon Telegraph and Telephone Corporation,227000
5,Samsung Electronics,221726
6,Accenture,205000
7,Tata Consultancy Services,200300
8,Flextronics International,200000
9,Safeway,186000


### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [15]:
# Your Code

In [16]:
def founded_twothousands():
    query=db.companies.find({'$and':[{'founded_year': {'$gte': 2000}},{'founded_year': {'$lte': 2006}}]} ,    
                            {'name': 1, '_id':0, 'founded_year':1}
                           )
    
    df=pd.json_normalize(data=query, meta=['name', 'founded_year'])
    
    return df

In [17]:
founded_twothousands().head()

,name,founded_year
0,Wetpaint,2005
1,Zoho,2005
2,Digg,2004
3,Facebook,2004
4,Omnidrive,2005


### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [18]:
# Your Code

In [19]:
def founded_2010_and_VA_more100M():
    query=db.companies.find({'$and':[{'founded_year': {'$lte': 2010}},{'ipo.valuation_amount': {'$gt': 100000000}}]} ,    
                            {'name': 1, '_id':0, 'ipo.valuation_amount':1}
                           )
    
    df=pd.json_normalize(data=query, meta=['name', 'ipo.valuation_amount'])
    
    return df

In [20]:
founded_2010_and_VA_more100M()

,name,ipo.valuation_amount
0,Facebook,104000000000
1,Twitter,18100000000
2,Yelp,1300000000
3,LinkedIn,9310000000
4,Amazon,100000000000
5,Brightcove,290000000
6,KIT digital,235000000
7,Nielsen,1600000000
8,OpenTable,1050000000
9,ChannelAdvisor,287000000


### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [21]:
# Your Code

In [22]:
def founded_before2005_and_less_1kemployees():
    query=db.companies.find({'$and':[{'founded_year': {'$lte': 2005}},{'number_of_employees': {'$gt': 1000}}]} ,    
                            {'name': 1, '_id':0, 'number_of_employees':1}
                           ).sort('number_of_employees', -1).limit(10)
    
    df=pd.json_normalize(data=query, meta=['name', 'number_of_employees'])
    
    return df

In [23]:
founded_before2005_and_less_1kemployees()

,name,number_of_employees
0,Siemens,405000
1,IBM,388000
2,Toyota,320000
3,PayPal,300000
4,Nippon Telegraph and Telephone Corporation,227000
5,Samsung Electronics,221726
6,Accenture,205000
7,Tata Consultancy Services,200300
8,Flextronics International,200000
9,Safeway,186000


### 6. All the companies that don't include the `partners` field.

In [24]:
# Your Code

In [25]:
def no_parnters_field():
    query=collection.find({'partners': {'$ne': 1}},    
                          {'name': 1, '_id':0}
                         )
    
    df=pd.json_normalize(data=query, meta=['name'])
    
    return df

In [26]:
no_parnters_field().head()

,name
0,Wetpaint
1,AdventNet
2,Zoho
3,Digg
4,Facebook


### 7. All the companies that have a null type of value on the `category_code` field.

In [27]:
# Your Code

In [28]:
def no_value_category_code():
    query=collection.find({'category_code': None},    
                          {'name': 1, '_id':0}
                         )
    
    df=pd.json_normalize(data=query, meta=['name'])
    
    return df

In [29]:
no_value_category_code().head()

,name
0,Collective
1,Snimmer
2,KoolIM
3,Level9 Media
4,VidKing


### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [30]:
# Your Code

In [31]:
def employees_less100_more1000():
    query=db.companies.find({'$and':[{'number_of_employees': {'$gte': 100}},{'number_of_employees': {'$lte': 999}}]} ,    
                            {'name': 1, '_id':0, 'number_of_employees':1}
                           )
    
    df=pd.json_normalize(data=query, meta=['name', 'number_of_employees'])
    
    return df

In [32]:
employees_less100_more1000().head()

,name,number_of_employees
0,AdventNet,600
1,AddThis,120
2,OpenX,305
3,LifeLock,644
4,Jajah,110


### 9. Order all the companies by their IPO price in a descending order.

In [33]:
# Your Code

In [34]:
def companies_by_IPO_price():
    query=collection.find({'ipo.valuation_amount': {'$ne': None}},    
                          {'name': 1, '_id':0, 'ipo.valuation_amount':1}
                         ).sort('ipo.valuation_amount', -1).limit(10)
    
    df=pd.json_normalize(data=query, meta=['name', 'ipo.valuation_amount'])
    
    return df

In [35]:
companies_by_IPO_price()

,name,ipo.valuation_amount
0,GREE,108960000000
1,Facebook,104000000000
2,Amazon,100000000000
3,Twitter,18100000000
4,Groupon,12800000000
5,Tencent,11000000000
6,Western Digital,9430000000
7,LinkedIn,9310000000
8,BMC Software,6000000000
9,Rackspace,5440000000


### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [36]:
# Your Code

In [37]:
def more_employees():
    query=db.companies.find({'number_of_employees': {'$gt': 0}},    
                            {'name': 1, '_id':0, 'number_of_employees':1}
                           ).sort('number_of_employees', -1).limit(10)
    
    df=pd.json_normalize(data=query, meta=['name', 'number_of_employees'])
    
    return df

In [38]:
more_employees()

,name,number_of_employees
0,Siemens,405000
1,IBM,388000
2,Toyota,320000
3,PayPal,300000
4,Nippon Telegraph and Telephone Corporation,227000
5,Samsung Electronics,221726
6,Accenture,205000
7,Tata Consultancy Services,200300
8,Flextronics International,200000
9,Safeway,186000


### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [39]:
# Your Code

In [40]:
def founded_second_semester():
    query=db.companies.find({'founded_month': {'$gte': 7 }},    
                            {'name': 1, '_id':0, 'founded_month':1}
                           )
    
    df=pd.json_normalize(data=query, meta=['name', 'founded_month']).loc[:1000]
    
    return df

In [41]:
founded_second_semester().head()

,name,founded_month
0,Wetpaint,10
1,Zoho,9
2,Digg,10
3,Omnidrive,11
4,eBay,9


### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [42]:
# Your Code

In [43]:
def founded_before2000_acquisition_amount_more10M():
    query=db.companies.find({'$and':[{'founded_year': {'$lte': 2000}},{'acquisition.price_amount': {'$gt': 10000000}}]} ,    
                            {'name': 1, '_id':0, 'acquisition.price_amount':1}
                           ).sort('acquisition.price_amount',-1)
    
    df=pd.json_normalize(data=query, meta=['name', 'acquisition.price_amount'])
    
    return df



In [44]:
founded_before2000_acquisition_amount_more10M().head()

,name,acquisition.price_amount
0,BEA Systems,8500000000
1,Navteq,8100000000
2,Sun Microsystems,7400000000
3,Pixar,7400000000
4,LSI,6600000000


### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [45]:
# Your Code

In [46]:
def acquired_after2010():
    query=db.companies.find({'acquisition.acquired_year': {'$gte': 2010}} ,    
                            {'name': 1, '_id':0, 'acquisition.price_amount':1}
                           ).sort('acquisition.price_amount',-1)
    
    df=pd.json_normalize(data=query, meta=['name', 'acquisition.price_amount'])
    
    return df

In [47]:
acquired_after2010().head()

,name,acquisition.price_amount
0,T-Mobile,3.900000e+10
1,Goodrich Corporation,1.840000e+10
2,LSI,6.600000e+09
3,National Semiconductor,6.500000e+09
4,Sybase,5.800000e+09


### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [48]:
# Your Code

In [49]:
def order_foundation():
    query=db.companies.find({'founded_year': {'$gte': 1 }},    
                            {'name': 1, '_id':0, 'founded_year':1}
                           ).sort('founded_year', -1)
    
    df=pd.json_normalize(data=query, meta=['name', 'founded_year'])
    
    return df

In [50]:
print(order_foundation().head())
order_foundation().tail()

        name  founded_year
0      Fixya          2013
1      Wamba          2013
2  Advaliant          2013
3       Fluc          2013
4     iBazar          2013


,name,founded_year
13131,Bachmann Industries,1833
13132,DuPont,1802
13133,Alstrasoft,1800
13134,SmallWorlds,1800
13135,US Army,1800


### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [51]:
# Your Code

In [52]:
def founded_first7days_by_acquisition_price():
    query=db.companies.find({'founded_day': {'$lte': 7 }} ,    
                            {'name': 1, '_id':0, 'acquisition.price_amount':1}
                           ).sort('acquisition.price_amount',-1).limit(10)
    
    df=pd.json_normalize(data=query, meta=['name', 'acquisition.price_amount'])
    
    return df

In [53]:
founded_first7days_by_acquisition_price()

,name,acquisition.price_amount
0,Netscape,4200000000
1,PayPal,1500000000
2,Zappos,1200000000
3,Alibaba,1000000000
4,Postini,625000000
5,Danger,500000000
6,Clearwell Systems,410000000
7,PrimeSense,345000000
8,Amobee,321000000
9,BlueLithium,300000000


### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [54]:
# Your Code

In [55]:
def web_more_4kemployees():
    query=db.companies.find({'$and':[{'category_code':'web'},{'number_of_employees': {'$gt':4000}}]} ,    
                            {'name': 1, '_id':0, 'number_of_employees':1}
                           ).sort('number_of_employees',1)
    
    df=pd.json_normalize(data=query, meta=['name', 'number_of_employees'])
    
    return df


In [56]:
web_more_4kemployees()

,name,number_of_employees
0,Expedia,4400
1,AOL,8000
2,Webkinz,8657
3,Rakuten,10000
4,Los Angeles Times Media Group,10000
5,Groupon,10000
6,Yahoo!,13600
7,eBay,15000
8,Experian,15500


### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [57]:
# Your Code

In [58]:
def acquisition_more10MEUR():
    query=db.companies.find({'$and':[{'acquisition.price_currency_code':'EUR'} ,{'acquisition.price_amount': {'$gte': 10000000}}]},    
                            {'name': 1, '_id':0, 'acquisition.price_amount':1}
                           )
    
    df=pd.json_normalize(data=query, meta=['name', 'acquisition.price_amount'])
    
    return df

In [59]:
acquisition_more10MEUR()

,name,acquisition.price_amount
0,ZYB,31500000
1,Apertio,140000000
2,Greenfield Online,40000000
3,Webedia,70000000
4,Wayfinder,24000000
5,Tuenti Technologies,70000000
6,BioMed Central,43400000


### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [60]:
# Your Code

In [61]:
def founded_first_trimester():
    query=db.companies.find({'acquisition.acquired_month': {'$lte': 3 }},    
                            {'name': 1, '_id':0, 'acquisition.price_amount':1}
                           )
    
    df=pd.json_normalize(data=query, meta=['name', 'acquisition.price_amount'])
    
    return df

In [62]:
founded_first_trimester().head(10)

,name,acquisition.price_amount
0,Kyte,NaN
1,NetRatings,327000000.0
2,blogTV,NaN
3,Livestream,NaN
4,iContact,169000000.0
5,Coghead,NaN
6,Dailymotion,168000000.0
7,Netvibes,NaN
8,Flickr,NaN
9,BabyCenter,NaN


# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [63]:
# Your Code

In [64]:
def founded_twothousands2():
    query=db.companies.find({'$and':[{'founded_year': {'$gte': 2000}},{'founded_year': {'$lte': 2010}},{'acquisition.acquired_year': {'$gte': 2010}}]} ,    
                            {'name': 1, '_id':0, 'founded_year':1}
                           )
    
    df=pd.json_normalize(data=query, meta=['name', 'founded_year'])
    
    return df

In [65]:
founded_twothousands2().head()

,name,founded_year
0,Wetpaint,2005
1,Digg,2004
2,Geni,2006
3,Kyte,2006
4,Veoh,2004


### 20. All the companies that have been 'deadpooled' after the third year.

In [66]:
# Your Code

In [67]:
def deadpool_after_thirdyear():
    query=db.companies.find({'deadpooled_year': {'$gt': 3 }},    
                            {'name': 1, '_id':0, 'deadpooled_year':1}
                           )
    
    df=pd.json_normalize(data=query, meta=['name', 'deadpooled_year'])
    
    return df

In [68]:
deadpool_after_thirdyear().head()

,name,deadpooled_year
0,Omnidrive,2008
1,Babelgum,2013
2,Sparter,2008
3,Thoof,2013
4,Mercora,2008
